In [9]:
# imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import itertools
import random
import io
import requests

from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import manhattan_distances, euclidean_distances
from mpl_toolkits.mplot3d import Axes3D

### Load data

In [13]:
cities_ds = pd.read_csv('https://raw.githubusercontent.com/zhangchenbocalvin/city-search-tool/main/data/cities_ds.csv')
people_ds = pd.read_csv('https://raw.githubusercontent.com/zhangchenbocalvin/city-search-tool/main/data/people_ds_10.csv')
people = people_ds['Weights'].str.replace('(', '', regex=True).str.replace(')', '', regex=True).str.replace(',', '', regex=True).str.split(expand=True)

people.head(20)

,0,1,2,3,4,5
0,0,0,0,0,0,0
1,0,0,0,0,0,1
2,0,0,0,0,0,2
3,0,0,0,0,0,3
4,0,0,0,0,1,0
5,0,0,0,0,1,1
6,0,0,0,0,1,2
7,0,0,0,0,1,3
8,0,0,0,0,2,0
9,0,0,0,0,2,1


### Add your preferecies about Purchase Power, Health Care, Pollution, QoL_H, Crime Rating and Unesco



In [14]:
df = pd.DataFrame([1, 1, 0, 1, 1, 1]).T
df

,0,1,2,3,4,5
0,1,1,0,1,1,1


In [15]:
people_ds['md'] = pd.DataFrame(manhattan_distances(df, people))

ValueError: Wrong number of items passed 40960, placement implies 1

In [ ]:
topk_people=pd.DataFrame(people_ds.sort_values('md').head(20)['City'].unique(),columns={'City'})

In [ ]:
topk_people

In [ ]:
topk_people_city = pd.merge(topk_people['City'],
                 cities_ds[['City','Purchase Power', 'Health Care', 'Pollution', 'QoL_H', 'Crime Rating', 'Unesco']],
                 on='City')
topk_people_city

In [ ]:
topk_city = pd.DataFrame(euclidean_distances(topk_people_city[['Purchase Power', 'Health Care', 'Pollution', 'QoL_H', 'Crime Rating', 'Unesco']], cities_ds[['Purchase Power', 'Health Care', 'Pollution', 'QoL_H', 'Crime Rating', 'Unesco']])).T
topk_city['mean']=topk_city.mean(axis=1)
city_Score=pd.concat([cities_ds['City'], topk_city['mean']], axis=1)
city_Score.sort_values('mean').head(10)

In [ ]:
print(city_Score.sort_values('mean')["City"][:10])

In [ ]:
df = cities_ds

rndperm = np.random.permutation(df.shape[0])
df_subset = df.loc[rndperm,:].copy()
data_subset = df_subset[['Purchase Power', 'Health Care', 'Pollution', 'Crime Rating', 'Unesco', 'QoL_H']].values


tsne = TSNE(n_components=2, verbose=1, perplexity=15, n_iter=800)
tsne_results = tsne.fit_transform(data_subset)

In [ ]:
new_dataframe = df_subset
new_dataframe['tsne-2d-one'] = tsne_results[:,0]
new_dataframe['tsne-2d-two'] = tsne_results[:,1]
new_dataframe["Score"] = np.zeros(len(new_dataframe))
topk_cities = city_Score.sort_values('mean')["City"][:10].values
new_dataframe["City"] = df_subset["City"]
for i in range(len(new_dataframe["City"])):
    if new_dataframe["City"][i] == topk_cities[0]:
        new_dataframe["Score"][i] = "Top"
    elif new_dataframe["City"][i] in topk_cities:
        new_dataframe["Score"][i] = "Recommended"
    else:
        new_dataframe["Score"][i] = "Others"
        

'''
plt.figure(figsize=(16,10))
plt.plot(tsne_results[:,0], tsne_results[:, 1], marker = 'o', linestyle='none')

'''
# Create an array with the colors you want to use
colors = ["#EA2C03", "#EADB03", "#0396EA"]
# Set your custom color palette
customPalette = sns.set_palette(sns.color_palette(colors))
sns.scatterplot(
    x="tsne-2d-one", y="tsne-2d-two",
    hue = "Score",
    palette=colors,
    data=new_dataframe,
    legend="full",
    alpha=0.3)